# Library Imports

In [1]:
import pandas as pd
import glob
import requests
import time

In [2]:
import seaborn as sns

In [3]:
from credentials import SPOTIFY_API_CLIENT_ID, SPOTIFY_API_CLIENT_SECRET, GENIUS_ACCESS_TOKEN, MUSIXMATCH_API_KEY

# How to get the data? and which data I'm using?

<div style="float: left">
<img style="float: right" src="datasets/privacy_page.jpg" alt="screenshot" width="600"/>
Good question! I actually just found this out recently myself that you can request your streaming data from Spotify! 

The process is quite simple, All you need to do is to access [your account's privacy page](https://www.spotify.com/us/account/privacy/) , put a tick on the data that you want to request from spotify (see the screenshot on the right)

The data I'm using in this project is the streaming hisotry (from Extended Streaming History option) where it consists of multiple .json files of your spotify streaming history (for both music and podcasts)
I chose the Extended Streaming History because the data includes your entire lifetime of your account, and it also includes _spotify_track_uri_, a unique identifier for each spotify tracks that we can use to collect even more information about the tracks (such as audio features, track info, etc) using [Spotify Web API](https://developer.spotify.com/) later.

Account Data only conists of one-year length of data and it does not have the _spotify_track_uri_ information, technically you can still collect the tracks' info, audio features data using only artists' name and tracks' name but it adds more work and can be quite unreliable (plus, extended streaming history gives you MORE data, so why not?)

Based on my experience, it took about 10 days to receive the Account Data and about 3 weeks to get Extended Streaming History from the time I requested the data.
</div>


# Data Imports / Exports

In [23]:
# THESE FILES ARE EXCLUDED FROM THE REPOSITORY
stream_json = glob.glob('datasets/v2/Streaming_History_Audio*.json')

In [24]:
# see the list of globbed files
stream_json

['datasets/v2\\Streaming_History_Audio_2016-2017_0.json',
 'datasets/v2\\Streaming_History_Audio_2017-2018_1.json',
 'datasets/v2\\Streaming_History_Audio_2018-2019_2.json',
 'datasets/v2\\Streaming_History_Audio_2019-2020_3.json',
 'datasets/v2\\Streaming_History_Audio_2020-2021_4.json',
 'datasets/v2\\Streaming_History_Audio_2021-2022_5.json',
 'datasets/v2\\Streaming_History_Audio_2022-2023_6.json',
 'datasets/v2\\Streaming_History_Audio_2023_7.json',
 'datasets/v2\\Streaming_History_Audio_2023_8.json']

In [25]:
# load the globbed files into a dataframe
stream_df = pd.concat([pd.read_json(f) for f in stream_json])

In [9]:
tracks_df = pd.read_csv('datasets/tracks_df.csv')
podcasts_df = pd.read_csv('datasets/podcasts_df.csv')

C:\Users\nazhi\AppData\Local\Temp\ipykernel_5344\2056156176.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks_df = pd.read_csv('datasets/tracks_df.csv')


# Preparing for Data Collection
we need 4 things here.
1. tracks streaming history
2. podcasts streaming history
3. list of unique tracks streamed
4. ~~list of unique podcasts streamed~~ not really needed actually

In [26]:
print(stream_df.shape)
print(stream_df.columns)

(130209, 21)
Index(['ts', 'username', 'platform', 'ms_played', 'conn_country',
       'ip_addr_decrypted', 'user_agent_decrypted',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'reason_start',
       'reason_end', 'shuffle', 'skipped', 'offline', 'offline_timestamp',
       'incognito_mode'],
      dtype='object')


In [27]:
stream_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130209 entries, 0 to 2979
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   ts                                 130209 non-null  object
 1   username                           130209 non-null  object
 2   platform                           130209 non-null  object
 3   ms_played                          130209 non-null  int64 
 4   conn_country                       130209 non-null  object
 5   ip_addr_decrypted                  109601 non-null  object
 6   user_agent_decrypted               109601 non-null  object
 7   master_metadata_track_name         129393 non-null  object
 8   master_metadata_album_artist_name  129393 non-null  object
 9   master_metadata_album_album_name   129393 non-null  object
 10  spotify_track_uri                  129393 non-null  object
 11  episode_name                       641 non-null     object


In [28]:
# number of unique values in each column
stream_df.nunique()

ts                                   122712
username                                  1
platform                                 33
ms_played                             27030
conn_country                              9
ip_addr_decrypted                      2750
user_agent_decrypted                      6
master_metadata_track_name             4783
master_metadata_album_artist_name      1646
master_metadata_album_album_name       2700
spotify_track_uri                      5245
episode_name                            297
episode_show_name                        31
spotify_episode_uri                     297
reason_start                              8
reason_end                               10
shuffle                                   2
skipped                                   2
offline                                   2
offline_timestamp                    128332
incognito_mode                            2
dtype: int64

In [29]:
stream_df.isna().sum()

ts                                        0
username                                  0
platform                                  0
ms_played                                 0
conn_country                              0
ip_addr_decrypted                     20608
user_agent_decrypted                  20608
master_metadata_track_name              816
master_metadata_album_artist_name       816
master_metadata_album_album_name        816
spotify_track_uri                       816
episode_name                         129568
episode_show_name                    129568
spotify_episode_uri                  129568
reason_start                              0
reason_end                            20608
shuffle                                   0
skipped                              106195
offline                                   0
offline_timestamp                         0
incognito_mode                            0
dtype: int64

In [30]:
stream_df['user_agent_decrypted'].value_counts()

user_agent_decrypted
unknown                                                                                                                                                                  109525
Mozilla%2F5.0%20(Windows%20NT%2010.0;%20Win64;%20x64)%20AppleWebKit%2F537.36%20(KHTML,%20like%20Gecko)%20Chrome%2F95.0.4638.54%20Safari%2F537.36%20Edg%2F95.0.1020.40        66
Mozilla%2F5.0%20(Windows%20NT%2010.0;%20Win64;%20x64;%20rv:84.0)%20Gecko%2F20100101%20Firefox%2F84.0                                                                          6
Mozilla%2F5.0%20(Windows%20NT%2010.0;%20Win64;%20x64;%20rv:61.0)%20Gecko%2F20100101%20Firefox%2F61.0                                                                          2
Mozilla%2F5.0%20(Windows%20NT%2010.0;%20Win64;%20x64;%20rv:69.0)%20Gecko%2F20100101%20Firefox%2F69.0                                                                          1
Mozilla%2F5.0%20(Windows%20NT%2010.0;%20Win64;%20x64;%20rv:83.0)%20Gecko%2F20100101%20Firefox%2F83.

In [31]:
# dropping some columns that might cnotain sensitive information or just not useful
stream_df.drop(columns=['username', 'ip_addr_decrypted', 'user_agent_decrypted'], inplace=True)

there are a huge number of null value of 'spotify_episode_uri', indicating that tracks/songs and podcasts don't share same uri, we need to separate this into its own dataframe (for tracks and podcasts)

In [32]:
# rows with null in both spotify_track_uri and spotify_episode_uri
print(stream_df[stream_df['spotify_episode_uri'].isna() & stream_df['spotify_track_uri'].isna()].shape)
print(stream_df[stream_df['spotify_episode_uri'].notna() & stream_df['spotify_track_uri'].notna()].shape)

(175, 18)
(0, 18)


so apparently there exists some rows are null on both spotify_episode_uri and spotify_track_uri, so those should be excluded as well

In [33]:
# get rows with null values in 'spotify_episode_uri' column and put it into a new dataframe
tracks_df = stream_df[stream_df['spotify_track_uri'].notna()]
podcasts_df = stream_df[stream_df['spotify_episode_uri'].notna()]

In [34]:
print(tracks_df.shape)
print(podcasts_df.shape)

(129393, 18)
(641, 18)


In [10]:
# list of tracks, containing unique values in 'spotify_track_uri' column
tracks_uri_list = tracks_df['spotify_track_uri'].unique().tolist()
podcasts_uri_list = podcasts_df['spotify_episode_uri'].unique().tolist()

In [11]:
print(len(tracks_uri_list))
print(len(podcasts_uri_list))

5245
297


In [37]:
# save tracks_df and podcasts_df into csv
tracks_df.to_csv('datasets/tracks_df.csv', index=False)
podcasts_df.to_csv('datasets/podcasts_df.csv', index=False)

# Prepping for Data Collection with Spotipy
now we're going to collect the using [Spotipy](https://spotipy.readthedocs.io/en/2.22.1/), it'll require user credentials (client_id and client_secret) which you can get [here](https://developer.spotify.com/dashboard) by registering your app at the dashboard

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Spotify API credentials (client_id and client_secret used will be excluded from the repository)
client_id = SPOTIFY_API_CLIENT_ID
client_secret = SPOTIFY_API_CLIENT_SECRET

In [5]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Collecting Track Info

In [12]:
# since they have the batch request limit of 50, we need to split the list into batches of 50
track_info = {}
for i in range(0, len(tracks_uri_list), 50):
    track_batch = tracks_uri_list[i:i+50]
    sapi = sp.tracks(track_batch)
    for track in sapi['tracks']:
        track_info[track['id']] = {'name':track['name'], 'artistName':track['artists'][0]['name'], 'release_date':track['album']['release_date'], 'popularity':track['popularity'], 'duration_ms':track['duration_ms'], 'isrc':track['external_ids'].get('isrc', None)}
    time.sleep(1)
    

In [6]:
sp.track('6tAM5c0bJOwRqGAEgiNMpI')

{'album': {'album_type': 'compilation',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
    'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
    'id': '0LyfQWJT6nXafLPZqxe9Of',
    'name': 'Various Artists',
    'type': 'artist',
    'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/7dArkHuO5wkus2TJTu6Ytx'},
  'href': 'https://api.spotify.com/v1/albums/7dArkHuO5wkus2TJTu6Ytx',
  'id': '7dArkHuO5wkus2TJTu6Ytx',
  'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b273da7770b79a66bc42de93d8e7',
    'width': 640,
    'height': 640},
   {'url': 'https://i.scdn.co/image/ab67616d00001e02da7770b79a66bc42de93d8e7',
    'width': 300,
    'height': 300},
   {'url': 'https://i.scdn.co/image/ab67616d00004851da7770b79a66bc42de93d8e7',
    'width': 64,
    'height': 64}],
  'name': 'Monstercat 007 - Solace',
  'release_date': '2012-

In [13]:
len(track_info)

5245

In [14]:
list(track_info.items())[:5]

[('6tAM5c0bJOwRqGAEgiNMpI',
  {'name': 'Solace Album Mix',
   'artistName': 'Monstercat',
   'release_date': '2012-06-06',
   'popularity': 0,
   'duration_ms': 3538579,
   'isrc': 'TCABG1284815'}),
 ('4mjgNE8R31AzxWfPNGtVMf',
  {'name': 'Best of 2015 (Album Mix)',
   'artistName': 'Monstercat',
   'release_date': '2016-01-22',
   'popularity': 0,
   'duration_ms': 9158194,
   'isrc': 'CA6D21600007'}),
 ('5Q0P0cX3e42PgKd8LLS3ms',
  {'name': 'Horizon Album Mix',
   'artistName': 'Monstercat',
   'release_date': '2014-08-06',
   'popularity': 0,
   'duration_ms': 3623121,
   'isrc': 'TCABY1492585'}),
 ('6jvMmRtSzoEibQGrQkSISQ',
  {'name': 'Monstercat Best of 2012',
   'artistName': 'Monstercat',
   'release_date': '2013-02-04',
   'popularity': 0,
   'duration_ms': 6348017,
   'isrc': 'TCABL1374362'}),
 ('1KzLyjpjIRHuuj4iX8QsC2',
  {'name': 'Monstercat Podcast EP. 100',
   'artistName': 'Monstercat',
   'release_date': '2016-04-05',
   'popularity': 0,
   'duration_ms': 9744610,
   'isrc

### Convert the dict to dataframe and save it to csv

In [15]:
track_info_df = pd.DataFrame.from_dict(track_info, orient='index')

In [16]:
track_info_df.to_csv('datasets/tracks_info_df_isrc.csv')

In [ ]:
# show rows with duplicate values in 'isrc' column
track_info_df[track_info_df['isrc'].duplicated(keep=False)].sort_values('isrc')


## Collecting Track Audio Features

In [46]:
track_features = {}
for i in range(0, len(tracks_uri_list), 50):
    track_batch = tracks_uri_list[i:i+50]
    sapi = sp.audio_features(track_batch)
    for track in sapi:
        if track is not None:
            track_features[track['id']] = {'danceability':track['danceability'], 'energy':track['energy'], 'key':track['key'], 'loudness':track['loudness'], 'mode':track['mode'], 'speechiness':track['speechiness'], 'acousticness':track['acousticness'], 'instrumentalness':track['instrumentalness'], 'liveness':track['liveness'], 'valence':track['valence'], 'tempo':track['tempo'], 'time_signature':track['time_signature']}
        # we'll skip it if the track has no audio features data
    time.sleep(1)

In [47]:
len(track_features)

5238

In [48]:
# get first 5 items in the dictionary
list(track_features.items())[:5]

[('6tAM5c0bJOwRqGAEgiNMpI',
  {'danceability': 0.446,
   'energy': 0.823,
   'key': 11,
   'loudness': -5.279,
   'mode': 0,
   'speechiness': 0.0859,
   'acousticness': 0.00256,
   'instrumentalness': 0.0748,
   'liveness': 0.117,
   'valence': 0.263,
   'tempo': 128.198,
   'time_signature': 4}),
 ('5Q0P0cX3e42PgKd8LLS3ms',
  {'danceability': 0.402,
   'energy': 0.856,
   'key': 1,
   'loudness': -4.256,
   'mode': 1,
   'speechiness': 0.0659,
   'acousticness': 0.00504,
   'instrumentalness': 0.0164,
   'liveness': 0.253,
   'valence': 0.247,
   'tempo': 140.028,
   'time_signature': 4}),
 ('4VrdksXJVhAOLW49qV0VTQ',
  {'danceability': 0.426,
   'energy': 0.915,
   'key': 11,
   'loudness': -3.881,
   'mode': 0,
   'speechiness': 0.143,
   'acousticness': 0.000899,
   'instrumentalness': 0.108,
   'liveness': 0.495,
   'valence': 0.271,
   'tempo': 130.405,
   'time_signature': 4}),
 ('0ng42pTjKgskmobNzhnEUa',
  {'danceability': 0.434,
   'energy': 0.839,
   'key': 8,
   'loudness': 

### Convert the dict to dataframe and save it to csv

In [49]:
track_features_df = pd.DataFrame.from_dict(track_features, orient='index')
track_features_df.to_csv('datasets/tracks_features_df.csv')

## Collecting Track Lyrics (using LyricsGenius / Genius API)

here, we are going to use [Genius API](https://genius.com/developers) through [LyricsGenius](https://github.com/johnwmillr/LyricsGenius) to get the lyrics of the tracks, you can get the API key by registering at the website.

In [20]:
import lyricsgenius
import re
import time

In [10]:
lyricsgenius_client = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)

In [7]:
tracks_info_df = pd.read_csv('datasets/tracks_info_df.csv')
print(tracks_info_df.shape)

(5245, 6)


In [8]:
tracks_info_df = tracks_info_df[~tracks_info_df.isna().any(axis=1)].drop_duplicates(subset='track_id')
print(tracks_info_df.shape)

(5243, 6)


In [37]:
tracks_info_df.head(5)

,track_id,name,artistName,release_date,popularity,duration_ms
0,6tAM5c0bJOwRqGAEgiNMpI,Solace Album Mix,Monstercat,2012-06-06,0,3538579
1,4mjgNE8R31AzxWfPNGtVMf,Best of 2015 (Album Mix),Monstercat,2016-01-22,0,9158194
2,5Q0P0cX3e42PgKd8LLS3ms,Horizon Album Mix,Monstercat,2014-08-06,0,3623121
3,6jvMmRtSzoEibQGrQkSISQ,Monstercat Best of 2012,Monstercat,2013-02-04,0,6348017
4,1KzLyjpjIRHuuj4iX8QsC2,Monstercat Podcast EP. 100,Monstercat,2016-04-05,0,9744610


In [ ]:
lyrics_data = {}
rate_limit_reached = False

# Loop through the list of unique isrc values
for i in range(95, len(tracks_info_df)):
    while True:
        try:
            song = lyricsgenius_client.search_song(tracks_info_df.iloc[i]['name'], tracks_info_df.iloc[i]['artistName'])
            track_id = tracks_info_df.iloc[i]['track_id']

            if song is not None:
                lyrics_data[track_id] = {
                    'title_genius': song.title,
                    'artist_genius': song.artist,
                    'title_spotify': tracks_info_df.iloc[i]['name'],
                    'artist_spotify': tracks_info_df.iloc[i]['artistName'],

                    # remove garbage character from ads in the lyrics
                    'lyrics': re.sub(r"(?<!\n)\n(\[)", r"\n\n\1", song.lyrics)
                }
            else:
                lyrics_data[track_id] = {
                    'title_genius': None,
                    'artist_genius': None,
                    'title_spotify': tracks_info_df.iloc[i]['name'],
                    'artist_spotify': tracks_info_df.iloc[i]['artistName'],
                    'lyrics': None
                }

            rate_limit_reached = False
            break  # Exit the retry loop if the request is successful

        except Exception as e:
            print(f"An error occurred: {e}")
            if not rate_limit_reached:
                print("Rate limit reached. Waiting for 60 seconds...")
                rate_limit_reached = True
            time.sleep(60)  # Wait for 60 seconds before retrying

# Save the data to a CSV file
lyrics_data_df = pd.DataFrame.from_dict(lyrics_data, orient='index')
lyrics_data_df.to_csv('datasets/lyrics_data.csv')

Searching for "Solace Album Mix" by Monstercat...
An error occurred: [Errno 429] 429 Client Error: Too Many Requests for url: https://genius.com/api/search/multi?q=Solace+Album+Mix+Monstercat
Rate limit reached. Waiting for 60 seconds...
